# 설정

In [64]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [65]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4tbur4yd
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4tbur4yd
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12773 sha256=1d5cbe619b5e8680af661b9c54d3737b7426685ab37984467bfb1f221c062fc5
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9c1f1zp/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [66]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [67]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [68]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [69]:
##GPU 사용 시
device = torch.device("cuda:0")

In [70]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [71]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-07-08 18:04:33--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-07-08 18:04:34--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0ea5061b71630bb8da1e0fff17.dl.dropboxusercontent.com/cd/0/get/BR4bnjTA1qrHE0FGL5FaEQnBnyJBKvpKxlmuMDt9Rpu6ym59eAOBU7xS6yFsuJFR0miOO_OZIRdTCVlnCY-kyW8cSntX3GtLL0twVXo9IrId6f-pXfFkiv3AmX8VGmmfhz9gV0WnWsSeBKFBt0Vywb3e/file?dl=1# [following]
--2021-07-08 18:04:34--  https://uc0ea5061b71630bb8da1e0fff17.dl.dropboxusercontent.com/cd/0/get/BR4bnjTA1qrHE0FGL5FaEQnBnyJBKvpKxlmuMDt9Rpu6ym59eAOBU7xS6yFsuJFR0miOO_OZI

# 모델

In [72]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [73]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [74]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [75]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [76]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [77]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [78]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)   # <-- return_dict=False 추가했습니다
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [79]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [80]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [81]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [82]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [83]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [84]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [85]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7534136176109314 train acc 0.421875
epoch 1 batch id 201 loss 0.4841904044151306 train acc 0.5683302238805971
epoch 1 batch id 401 loss 0.4691815972328186 train acc 0.6706281172069826
epoch 1 batch id 601 loss 0.42668336629867554 train acc 0.7217398086522463
epoch 1 batch id 801 loss 0.4014209806919098 train acc 0.7504291510611736
epoch 1 batch id 1001 loss 0.3169175386428833 train acc 0.7689185814185814
epoch 1 batch id 1201 loss 0.3968413472175598 train acc 0.7832535387177352
epoch 1 batch id 1401 loss 0.31889382004737854 train acc 0.7934957173447538
epoch 1 batch id 1601 loss 0.33917322754859924 train acc 0.8024476889444098
epoch 1 batch id 1801 loss 0.2576323449611664 train acc 0.8095415741254859
epoch 1 batch id 2001 loss 0.3136122226715088 train acc 0.8159357821089456
epoch 1 batch id 2201 loss 0.2863525152206421 train acc 0.8211040436165379

epoch 1 train acc 0.8246409271899886


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8850903132992327


...


epoch 3 test acc 0.8977581521739131


epoch 4 batch id 1 loss 0.4518713355064392 train acc 0.875
epoch 4 batch id 201 loss 0.0532354861497879 train acc 0.9549906716417911
epoch 4 batch id 401 loss 0.12313670665025711 train acc 0.955423940149626
epoch 4 batch id 601 loss 0.2603355646133423 train acc 0.9571027454242929
epoch 4 batch id 801 loss 0.1373017579317093 train acc 0.9583333333333334
epoch 4 batch id 1001 loss 0.1508369743824005 train acc 0.960118006993007
epoch 4 batch id 1201 loss 0.08379685133695602 train acc 0.9611781848459617
epoch 4 batch id 1401 loss 0.0765957459807396 train acc 0.9620360456816559
epoch 4 batch id 1601 loss 0.03298240900039673 train acc 0.9630016396002499
epoch 4 batch id 1801 loss 0.1057875007390976 train acc 0.9636833703498057
epoch 4 batch id 2001 loss 0.07018205523490906 train acc 0.9644630809595203
epoch 4 batch id 2201 loss 0.0740780159831047 train acc 0.9649520104497955

epoch 4 train acc 0.9655281214448236



epoch 4 test acc 0.8963794757033248


epoch 5 batch id 1 loss 0.4212789237499237 train acc 0.875
epoch 5 batch id 201 loss 0.06236227601766586 train acc 0.9748134328358209
epoch 5 batch id 401 loss 0.03308486193418503 train acc 0.9746726932668329
epoch 5 batch id 601 loss 0.14140084385871887 train acc 0.9746776206322796
epoch 5 batch id 801 loss 0.09755707532167435 train acc 0.9755383895131086
epoch 5 batch id 1001 loss 0.0658656507730484 train acc 0.9763829920079921
epoch 5 batch id 1201 loss 0.0726473331451416 train acc 0.9767641548709409
epoch 5 batch id 1401 loss 0.017929790541529655 train acc 0.9770253390435403
epoch 5 batch id 1601 loss 0.020812680944800377 train acc 0.9772993441599
epoch 5 batch id 1801 loss 0.021970340982079506 train acc 0.9775732232093282
epoch 5 batch id 2001 loss 0.011775418184697628 train acc 0.9778470139930034
epoch 5 batch id 2201 loss 0.16172200441360474 train acc 0.9780000567923671

epoch 5 train acc 0.9782289889078498



epoch 5 test acc 0.8974584398976982


In [ ]:
RELEVANT_LABELS = [0,1]
SCORE_LABELS = [-1,0,1,2]
model.eval()
relevant_true = []
relevant_pred = []

sentiment_true = []
sentiment_pred = []
for batch_id, (token_ids, valid_length, segment_ids, related, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  related = related.long().to(device)
  out1, out2 = model(token_ids, valid_length, segment_ids)
  _, related_index = torch.max(out1, 1)
  _, label_index = torch.max(out2, 1)
  for idx in range(len(label)):
    sentiment_true.append(S_LABELS[int(label[idx])])
    relevant_true.append(R_LABELS[int(related[idx])])
    sentiment_pred.append(S_LABELS[int(label_index[idx])])
    relevant_pred.append(R_LABELS[int(related_index[idx])])
print(sentiment_true)
print(sentiment_pred)
print(relevant_true)
print(relevant_pred)

from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter



print('da confusion matrix')
print('Labels: [-1, 0, 1, 2] (2 is not labeled because irrelevant)')
print(confusion_matrix(sentiment_true, sentiment_pred, labels=[-1,0,1,2]))
print(classification_report(sentiment_true, sentiment_pred, digits=3))



print('da confusion matrix')
print(confusion_matrix(relevant_true, relevant_pred, labels=[0,1]))
print(classification_report(relevant_true, relevant_pred, digits=3))